# New index for `Curve`

In [1]:
import numpy as np
import xarray as xr

/home/matt/anaconda3/envs/geocomp/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


### Multi-dimensional index, try 1

In [4]:
data = [3,9,5,4,3,4,4,4,3,8,5,6,7,]
md = np.arange(len(data)) + 100
tvd = np.arange(len(data))**1.05 + 50

In [5]:
md, tvd

(array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]),
 array([50.        , 51.        , 52.07052985, 53.16940193, 54.28709385,
        55.41899193, 56.56234129, 57.71530221, 58.87655578, 60.04510857,
        61.22018454, 62.40116025, 63.5875235 ]))

In [6]:
# This does not work.
# xmd = xr.DataArray(md, name='MD', dims=['TVD'], coords={'TVD': tvd})
# xdata = xr.DataArray(data, name='data', dims=['MD'], coords={'MD': xmd})

### Multi-dimensional index, try 2

In [9]:
depth = np.stack([md, tvd]).T

xdata = xr.DataArray(data, name='data', dims=['MD'], coords={'MD':md})

In [10]:
xdata['TVD'] = ('MD', tvd)

In [11]:
xdata

<xarray.DataArray 'data' (MD: 13)>
array([3, 9, 5, 4, 3, 4, 4, 4, 3, 8, 5, 6, 7])
Coordinates:
  * MD       (MD) int64 100 101 102 103 104 105 106 107 108 109 110 111 112
    TVD      (MD) float64 50.0 51.0 52.07 53.17 54.29 ... 60.05 61.22 62.4 63.59

In [12]:
xdata.interp(MD=102.5)

<xarray.DataArray 'data' ()>
array(4.5)
Coordinates:
    TVD      float64 52.62
    MD       float64 102.5

However, this does not work:

In [14]:
xdata.interp(TVD=52.5)

ValueError: dimensions ['TVD'] do not exist

### Swap dims

In [16]:
xdata = xdata.coords['MD'].swap_dims({'MD': 'TVD', })

In [18]:
xdata.interp(TVD=52.5)

<xarray.DataArray 'MD' ()>
array(102.390828)
Coordinates:
    MD       float64 102.4
    TVD      float64 52.5

YES!

But now this doesn't work:

In [20]:
xdata.interp(MD=102.5)

ValueError: dimensions ['MD'] do not exist

### MultiIndex

In [21]:
xdata = xdata.set_index({'depth':['MD','TVD']})
xdata

/home/matt/anaconda3/envs/geocomp/lib/python3.7/site-packages/xarray/core/dataset.py:167: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  idx = pd.MultiIndex(labels=labels, levels=levels, names=names)


<xarray.DataArray 'MD' (TVD: 13)>
array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112])
Coordinates:
  * depth    (depth) MultiIndex
  - MD       (depth) int64 100 101 102 103 104 105 106 107 108 109 110 111 112
  - TVD      (depth) float64 50.0 51.0 52.07 53.17 ... 60.05 61.22 62.4 63.59
Dimensions without coordinates: TVD

In [22]:
xdata.interp(depth=101)

<xarray.DataArray 'MD' (TVD: 13)>
array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112])
Coordinates:
    depth    int64 101
Dimensions without coordinates: TVD

In [23]:
xdata.interp(TVD=52.5)

<xarray.DataArray 'MD' ()>
array(nan)
Coordinates:
  * depth    (depth) MultiIndex
  - MD       (depth) int64 100 101 102 103 104 105 106 107 108 109 110 111 112
  - TVD      (depth) float64 50.0 51.0 52.07 53.17 ... 60.05 61.22 62.4 63.59
    TVD      float64 52.5

In [26]:
xdata.interp(MD=102.5)

ValueError: dimensions ['MD'] do not exist

In [25]:
xdata.interp(depth={'MD':102.5})

<xarray.DataArray 'MD' (TVD: 13)>
array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112])
Coordinates:
    depth    object {'MD': 102.5}
Dimensions without coordinates: TVD